<a href="https://colab.research.google.com/github/guoweihua0307/guoweihua0307/blob/main/ChatGPTDevClass5ChatDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Q&A over Documents 根据你的文档回答问题

让ChatGPT根据你提供的文档来回答问题，这个文档可以是公司产品介绍，客户FAQ等等

In [ ]:
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
file = 'Insta360Products.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
!pip install docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00


In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
query ="列出适合滑雪使用的产品"

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

 Insta360 ONE RS, Insta360 X3, Insta360 GO3.

In [ ]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

Document(page_content='\ufeff产品名: Insta360 X3\n产品描述: 视频拍摄能力：5.7K 30P，4K 60P，4K 30P，2.7K 60P，2.7K 30P，1080P 60P，1080P 30P，1080P 24P，1080P 25P，4K 24P，4K 25P；分辨率：5.7k；适用场景：骑行，滑雪，Vlog；功能：裸机防水，语音控制，触屏操作，视频直播，电子防抖，直播推流，360°全景；画幅：1/2.0英寸', metadata={'source': 'Insta360Products.csv', 'row': 0})

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("你好，我是史丹福机器人庞博士")

In [ ]:
print(len(embed))

1536


In [ ]:
print(embed[:4])

[-0.028203697875142097, -0.015613230876624584, -0.013142449781298637, -0.010231397114694118]


In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [ ]:
query = "列出适合滑雪使用的产品"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(page_content='\ufeff产品名: Insta360 ONE RS\n产品描述: 画幅：1/2.0英寸；适用场景：骑行，滑雪，Vlog；分辨率：5.7k；功能：裸机防水，语音控制，触屏操作，可更换镜头，全景广角两用', metadata={'source': 'Insta360Products.csv', 'row': 4})

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0)


In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} 问题：列出适合滑雪使用的产品。")


In [ ]:
display(Markdown(response))

适合滑雪使用的产品有：
1. Insta360 ONE RS
2. Insta360 X3
3. Insta360 GO3
4. Insta360 Flow

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
query =  "列出适合滑雪使用的产品"

In [ ]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
display(Markdown(response))

适合滑雪使用的产品有：

1. Insta360 ONE RS
2. Insta360 X3
3. Insta360 GO3
4. Insta360 Flow

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

适合滑雪使用的产品有：

1. Insta360 ONE RS
2. Insta360 X3
3. Insta360 GO3
4. Insta360 Flow

# 根据你的PDF文档来回答问题

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
# 你需要把PDF文件放到当前目录下
# Simple method - Split by pages 简单的方法： 把PDF按页分
loader = PyPDFLoader("./史丹福机器人云实习s.pdf")
pages = loader.load_and_split()
print(pages[0])


page_content='你在申请海外大学或研究院吗？你在美国大学要毕业找工作吗？只有不错的成绩是不够的，最重要的是要有一个亮眼的实习经历和世界顶尖大学毕业的企业家的推荐信，尤其你是要找人工智能、机器人、自动驾驶这些最热门的专业和工作。 近两年我已经为多位这样的学子安排到我们公司实习，根据他们的兴趣和特长规划实习的项目，最后有我给他们出推荐信，最终他们都拿到了心仪的美国大学的录取，进入了一流的美国大学研究院，和一毕业就在硅谷拿到了百万年薪的工作。 现在我们决定提供史丹福机器人云实习给更多需要的同学。 这个云实习由我和我的CTO： StanfordRobotics.AI史丹福机器人的两位三维视觉和VSLAM的世界顶级专家博士，亲自主导： 我本人，庞琳勇Leo博士，斯坦福大学机械工程博士和计算机硕士双学位，中科大本硕，师从光测泰斗伍小平院士，并提前一年毕业。我是机器人、三维视觉、双目和半导体的专家，有22年硅谷和国内创业，NASDAQ上市，公司行政管理、市场和营销的经验。其中1家创业公司在NASDAQ上市，主导了其上一家创业家公司的收购。我是小觅智能MYNT AI和史丹福机器人的创始人。拥有38项专利，还有28项待批，83篇论文在国际期刊和学术会议上发表。 张国铉Alan博士，韩国汉阳大学VSLAM博士，美国普林斯顿大学、新加波理工大学、美国Drexel大学 研究VSLAM的博士后。张博士是前美国Rainbow 机器人公司的VSLAM首席科学家（其机器人HUBO在2015年美国DARPA举办的全球机器人挑战赛决赛中击败了波士顿动力、和MIT，  \n  斯坦福，CMU等名校的机器人，获得冠军）。曾在硅谷创立Astar.AI，研发出了全球领先的基于物体边缘特征的鱼眼双目VSLAM系统，其核心专利VSLAM System已经获得美国专利局授权。  这个史丹福机器人的云实习收费2万美金，包括实习中的辅导、课程、课件、申请大学和申请工作辅导、以及我和张博士的推荐信。其中1万美金直到学生拿到他心仪的大学的录取或找到工作才收取，同时在这最终目标达成之前学生可以一直在云实习。 机会难得，名额有限，我们会择优录取。要报名可以直接微信给我leo_usa，并附上你的简历，我会亲自评估。 庞琳勇博士 史丹福机器人创始人&CEO \n\r\x0c\x08\x0e\x03\x11\t\x07\

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [ ]:
query =  "这个实习是多长时间？"

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

这个云实习不限时间，学生按照自己的时间和能力安排，直到他们拿到大学的录取或找到工作为止。